In [22]:
import os 

In [23]:
%pwd

'c:\\Users\\GEN-AI\\End-to-End-machine-project-with-mlflow-'

In [3]:
os.chdir("../")

In [24]:
%pwd

'c:\\Users\\GEN-AI\\End-to-End-machine-project-with-mlflow-'

In [53]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/dxsm6996/End-to-End-machine-project-with-mlflow-.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "dxsm6996"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "f90be35b72976c9fa0212395125c3e069ad1701d"



In [87]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationsConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [88]:
import sys
from pathlib import Path

# Add the parent directory of 'mlProject' to sys.path if not already present
project_root = Path().resolve().parent
if str(project_root) not in sys.path:
	sys.path.insert(0, str(project_root))

from mlProject.constants import *
from mlProject.utils.common import read_yaml,create_directories,save_json

In [89]:
class ConfigurationManeger:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evalution_config(self) -> ModelEvaluationsConfig:
        config = self.config.model_evaluations
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        model_evaluation_config = ModelEvaluationsConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/dxsm6996/End-to-End-machine-project-with-mlflow-.mlflow"
        )
        return model_evaluation_config



        

        

In [90]:
import os 
import pandas as pd
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score,mean_absolute_error
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib


In [91]:
""

''

In [96]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationsConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mse = mean_squared_error(actual, pred)
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mse, r2, mae

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)
            rmse, mse, r2, mae = self.eval_metrics(test_y, predicted_qualities)

            # Ensure directory exists before saving
            Path(self.config.metric_file_name).parent.mkdir(parents=True, exist_ok=True)

            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2, "mse": mse}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mse", mse)
            mlflow.log_metric("mae", mae)

            # model registry does not work with file store
            # if tracking_url_type_store != "file":
            #     mlflow.sklearn.log_model(model, "model")
            # else:
            #     mlflow.sklearn.log_model(model, "model")

In [97]:
try:
    config =ConfigurationManeger()
    model_evaluation_config = config.get_model_evalution_config()
    model_evaluation_config = ModelEvaluation(config =model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-07-02 12:04:11,087: INFO: common: YAML file config\config.yaml loaded successfully]
[2025-07-02 12:04:11,113: INFO: common: YAML file params.yaml loaded successfully]
[2025-07-02 12:04:11,123: INFO: common: YAML file schema.yaml loaded successfully]
[2025-07-02 12:04:11,132: INFO: common: Created directory at: artifacts]


[2025-07-02 12:04:11,709: WARNING: connectionpool: Retrying (Retry(total=6, connect=7, read=6, redirect=7, status=7)) after connection broken by 'ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)': /dxsm6996/End-to-End-machine-project-with-mlflow-.mlflow/api/2.0/mlflow/runs/create]
[2025-07-02 12:04:13,118: INFO: common: JSON file saved at: artifacts\model_evaluations\metrics.json]
🏃 View run thundering-snipe-296 at: https://dagshub.com/dxsm6996/End-to-End-machine-project-with-mlflow-.mlflow/#/experiments/0/runs/1c2a4c9270ac4671a043aa3f837c25fa
🧪 View experiment at: https://dagshub.com/dxsm6996/End-to-End-machine-project-with-mlflow-.mlflow/#/experiments/0
